In [43]:
import tensorflow as tf
import csv
import random
with open('.\Iris.csv') as csvfile:        
    csvreader = csv.reader(csvfile)
    next(csvreader, None) # skip header
    dataset = list(csvreader)
random.shuffle(dataset)
datatrain = dataset[1:int(len(dataset))]
data = [data for data in datatrain]
labels = [data[4] for data in datatrain]   


In [44]:
label_ref = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
train_set=[]
train_stances=[]
test_set=[]
test_stances=[]
for data in dataset:
    if random.randint(1,10)<2:
        test_set.append(data[1:5])
        test_stances.append(label_ref[data[5]])
    else:
        train_set.append(data[1:5])
        train_stances.append(label_ref[data[5]])

In [51]:
# Initialise hyperparameters
r = random.Random()
r.seed(123)
target_size = 3
hidden_size = 3

l2_alpha = 0.00001
learn_rate = 0.001
clip_ratio = 5
batch_size_train = 4
epochs = 90
n_train = len(train_set)#the total number of entry instances

#===============================
# Define model
#==============================

feature_size = len(train_set[0])

#to clear the existed nodes for new model training
tf.reset_default_graph() 

# Create placeholders
features_pl = tf.placeholder(tf.float32, [None, feature_size], 'features')
stances_pl = tf.placeholder(tf.int64, [None], 'stances')

# Infer batch size
batch_size = tf.shape(features_pl)[0]

# Define multi-layer perceptron
hidden_layer = tf.contrib.layers.fully_connected(features_pl, hidden_size,weights_initializer=tf.contrib.layers.xavier_initializer(seed=100))
logits_flat = tf.contrib.layers.linear(hidden_layer, target_size,weights_initializer=tf.contrib.layers.xavier_initializer(seed=102))
logits = tf.reshape(logits_flat, [batch_size, target_size])#reshape to be (batch_size*4)

# Define L2 loss
tf_vars = tf.trainable_variables()
l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tf_vars if 'bias' not in v.name]) * l2_alpha

# Define overall loss
loss = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=stances_pl) + l2_loss)

# Define prediction
softmaxed_logits = tf.nn.softmax(logits)
#predict = tf.argmax(softmaxed_logits, 1)
predict=softmaxed_logits
# Define optimiser
opt_func = tf.train.AdamOptimizer(learn_rate)
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tf_vars), clip_ratio)
opt_op = opt_func.apply_gradients(zip(grads, tf_vars))

# Perform training
with tf.Session() as sess:
    with tf.device("/cpu:0"	):
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()

        for epoch in range(epochs):
            total_loss = 0
            indices = list(range(n_train))
            r.shuffle(indices)

            for i in range(n_train // batch_size_train):
                batch_indices = indices[i * batch_size_train: (i + 1) * batch_size_train]
                batch_features = [train_set[i] for i in batch_indices]
                batch_stances = [train_stances[i] for i in batch_indices]

                batch_feed_dict = {features_pl: batch_features, stances_pl: batch_stances}
                _, current_loss = sess.run([opt_op, loss], feed_dict=batch_feed_dict)
                total_loss += current_loss

            #print('epoch',epoch,'loss',total_loss)

        #=====Save the Checkpoints===============================
        saver.save(sess, './model/mymodel')
        #=========================================================

        # Predict
        test_feed_dict = {features_pl: test_set}
        test_pred = sess.run(predict, feed_dict=test_feed_dict)       
print (test_pred)

[[9.84494865e-01 1.55000621e-02 5.00456599e-06]
 [1.62312377e-03 5.24277210e-01 4.74099725e-01]
 [1.54590361e-05 1.17823906e-01 8.82160604e-01]
 [9.72620428e-01 2.73569375e-02 2.26339598e-05]
 [8.07341561e-03 6.87351465e-01 3.04575086e-01]
 [1.99403777e-03 5.11100590e-01 4.86905336e-01]
 [9.86262083e-01 1.37342978e-02 3.59700834e-06]
 [5.75768854e-03 7.09211826e-01 2.85030484e-01]
 [1.18419842e-03 4.20621753e-01 5.78194141e-01]
 [9.73863840e-01 2.61178296e-02 1.83406883e-05]
 [1.19300021e-04 2.14296401e-01 7.85584271e-01]
 [1.65036842e-02 8.32567096e-01 1.50929227e-01]
 [1.47579401e-03 4.44780618e-01 5.53743541e-01]
 [1.95840355e-02 8.15351427e-01 1.65064558e-01]]


In [52]:

import numpy as np
final_pred_index = np.argmax(test_pred,1)
print(final_pred_index)

[0 1 2 0 1 1 0 1 2 0 2 1 2 1]


In [53]:
LABELS = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
cm = [[0, 0, 0 ],
          [0, 0, 0 ],
          [0, 0, 0],]
for i, (g, t) in enumerate(zip(final_pred_index, test_stances)):
    g_stance, t_stance = g, t
    cm[g_stance][t_stance] += 1
if sum(cm[0])==0:
        setosa=0
else:
    agree_recall=cm[0][0]/sum(cm[0])
if sum(cm[1])==0:
    versicolor=0
else:
    disagree_recall=cm[1][1]/sum(cm[1])
if sum(cm[2])==0:
    virginica=0
else:
    discuss_recall=cm[2][2]/sum(cm[2])

    
if sum(cm[0]+cm[1]+cm[2])==0:
    all_recall=0
else:
    all_recall=(cm[0][0]+cm[1][1]+cm[2][2])/sum(cm[0]+cm[1]+cm[2])
    lines = []
header = "|{:^15}|{:^15}|{:^15}|{:^15}|".format('', *LABELS)
line_len = len(header)
lines.append("-"*line_len)
lines.append(header)
lines.append("-"*line_len)

hit = 0
total = 0
for i, row in enumerate(cm):
    hit += row[i]
    total += sum(row)
    lines.append("|{:^15}|{:^15}|{:^15}|{:^15}|".format(LABELS[i],*row))
    lines.append("-"*line_len)
print('\n'.join(lines))
print('setosa',Agree)
print('versicolor',Disagree)
print('virginica ',Discuss)
print('virginica ',Unrelated)
print('Accuracy',Recall)

-----------------------------------------------------------------
|               |  Iris-setosa  |Iris-versicolor|Iris-virginica |
-----------------------------------------------------------------
|  Iris-setosa  |       4       |       0       |       0       |
-----------------------------------------------------------------
|Iris-versicolor|       0       |       6       |       0       |
-----------------------------------------------------------------
|Iris-virginica |       0       |       1       |       3       |
-----------------------------------------------------------------


NameError: name 'Agree' is not defined